In [ ]:
#default_exp EditOperations

In [ ]:
#export
#hide
from typing import List, Union, Tuple, Type

In [ ]:
#hide
import sys
sys.path.append("..")

from hephaestus.AbstractMethod import *

In [ ]:
#hide
from nbdev.showdoc import *

# EditOperations

> Defines various EditOperations which can be applied to an AbstractMethod to change its tokens.

In [ ]:
#export
class InsertOperation:
    """
    Creates an `InsertOperation` which defines the insertion of a token into an `AbstractMethod`. Applying the
    operation will insert token `newToken` at index `tokenIndex`.
    """

    def __init__(self, tokenIndex: int, newToken: str) -> None:
        self.__tokenIndex = tokenIndex
        self.__newToken = newToken
    
    def __str__(self) -> str:
        return "INSERT {} -> '{}'".format(self.__tokenIndex, self.__newToken)
    
    def __repr__(self) -> str:
        return str(self)

    def getIndex(self) -> int:
        """
        Returns the index of the insertion.
        """
        return self.__tokenIndex
    
    def getToken(self) -> str:
        """
        Returns the token to be inserted.
        """
        return self.__newToken
    
    def applyToTokens(self, tokens: List[str]) -> None:
        """
        Applies the insertion to the given list of tokens. Raises an error if the token index is out of bounds.
        """

        if self.__tokenIndex > len(tokens):
            raise IndexError("InsertOperation: cannot insert at index {} (out of bounds)".format(self.__tokenIndex))
        
        tokens.insert(self.__tokenIndex, self.__newToken)

In [ ]:
method = AbstractMethod("public void METHOD_1 ( ) { }")
method

public void METHOD_1 ( ) { }

In [ ]:
method.applyEditOperation(InsertOperation(1, "static"))
method

public static void METHOD_1 ( ) { }

In [ ]:
#export
class DeleteOperation:
    """
    Creates a `DeleteOperation` which defines the deletion of a token from an `AbstractMethod`. Applying the
    operation will delete the token at index `tokenIndex`.
    """

    def __init__(self, tokenIndex: int) -> None:
        self.__tokenIndex = tokenIndex
    
    def __str__(self) -> str:
        return "DELETE {}".format(self.__tokenIndex)

    def __repr__(self) -> str:
        return str(self)
    
    def getIndex(self) -> int:
        """
        Returns the index of the deletion.
        """
        return self.__tokenIndex
    
    def getToken(self) -> None:
        """
        Returns the token to be inserted. Since there is never an inserted token, return None.
        """
        return None
    
    def applyToTokens(self, tokens: List[str]) -> None:
        """
        Applies the deletion to the given list of tokens. Raises an error if the token index is out of bounds.
        """

        if self.__tokenIndex >= len(tokens):
            raise IndexError("DeleteOperation: cannot delete at index {} (out of bounds)".format(self.__tokenIndex))

        del tokens[self.__tokenIndex]

In [ ]:
method = AbstractMethod("public static void METHOD_1 ( ) { }")
method

public static void METHOD_1 ( ) { }

In [ ]:
method.applyEditOperation(DeleteOperation(1))
method

public void METHOD_1 ( ) { }

In [ ]:
#export
class ReplaceOperation:
    """
    Creates a `ReplaceOperation` which defines the replacement of a token in an `AbstractMethod`. Applying the
    operation will replace the token at index `tokenIndex` with token `newToken`.
    """

    def __init__(self, tokenIndex: int, newToken: str) -> None:
        self.__tokenIndex = tokenIndex
        self.__newToken = newToken
    
    def __str__(self) -> str:
        return "REPLACE {} -> '{}'".format(self.__tokenIndex, self.__newToken)

    def __repr__(self) -> str:
        return str(self)
    
    def getIndex(self) -> int:
        """
        Returns the index of the replacement.
        """
        return self.__tokenIndex
    
    def getToken(self) -> str:
        """
        Returns the replacing token.
        """
        return self.__newToken
    
    def applyToTokens(self, tokens: List[str]) -> None:
        """
        Applies the replacement to the given list of tokens. Raises an error if the token index is out of bounds.
        """

        if self.__tokenIndex >= len(tokens):
            raise IndexError("ReplaceOperation: cannot replace at index {} (out of bounds)".format(self.__tokenIndex))

        tokens[self.__tokenIndex] = self.__newToken

In [ ]:
method = AbstractMethod("public void METHOD_1 ( ) { }")
method

public void METHOD_1 ( ) { }

In [ ]:
method.applyEditOperation(ReplaceOperation(0, "private"))
method

private void METHOD_1 ( ) { }

In [ ]:
#export
#hide
# Define this here first with a forward reference to CompoundOperation
EditOperation = Union[InsertOperation, DeleteOperation, ReplaceOperation, "CompoundOperation"]

In [ ]:
#export
class CompoundOperation:
    """
    Creates a `CompoundOperation`, which represents a combination of EditOperations starting the given `operation`. More 
    operations can later be added. Applying the `CompoundOperation` to an `AbstractMethod` will apply all successfully
    added operations, in order.
    """

    def __init__(self, operation: EditOperation) -> None:
        """
        Creates the compound operation from the given operation.

        Members:

        `__beginIndex`: start of the range of deleted tokens
        `__endIndex`: end of the range of deleted tokens
        `__newTokens`: list of tokens to add
        `__type`: the type of the compound operation; can either be InsertOperation, DeleteOperation, or ReplaceOperation
        """
        
        # determine the range of deleted indices, will be nothing for an insert operation, or will span >= 1
        # token for any other type
        if type(operation) is CompoundOperation:
            self.__beginIndex, self.__endIndex = operation.getIndexRange()
        else:
            self.__beginIndex = operation.getIndex()
            self.__endIndex = self.__beginIndex if type(operation) is InsertOperation else self.__beginIndex + 1
        
        # determine new tokens which will be added, will be nothing for delete operation
        if type(operation) is CompoundOperation:
            self.__newTokens = operation.getTokens()
        else:
            self.__newTokens = [] if type(operation) is DeleteOperation else [operation.getToken()]
        
        # set the type
        self.__type = None
        self.__setType()
    
    def __str__(self) -> str:

        s =  "COMPOUND_{} ".format({
            InsertOperation: "INSERT",
            DeleteOperation: "DELETE",
            ReplaceOperation: "REPLACE"
        }[self.__type])
        
        if self.__type is InsertOperation:
            s += "{}".format(self.__beginIndex)
        else:
            s += "{}:{}".format(self.__beginIndex, self.__endIndex)
        
        if self.__type is not DeleteOperation:
            s += " -> {}".format(self.__newTokens)
        
        return s
    
    def __repr__(self) -> str:
        return str(self)
    
    def __len__(self) -> int:
        """
        Returns the Levenshtein distance from A to B, where A is some `AbstractMethod` and B is the `AbstractMethod`
        resulting from applying this compound operation to A.
        """
        return max(self.__endIndex - self.__beginIndex, len(self.__newTokens))
    
    def getIndexRange(self) -> Tuple[int, int]:
        """
        Returns the range of indices whose tokens are deleted as a tuple in the form [start, end).
        """
        return (self.__beginIndex, self.__endIndex)
    
    def getTokens(self) -> List[str]:
        """
        Returns the list of tokens to be added.
        """
        return self.__newTokens.copy()
    
    def getType(self) -> Type[EditOperation]:
        """
        Returns the type of the `CompoundOperation`, which can be one of three values:

        - `InsertOperation`: Applying this operation will only result in token insertions
        - `DeleteOperation`: Applying this operation will only result in token deletions
        - `ReplaceOperation`: Applying this operation will result in token deletions and insertions
        """
        return self.__type

    def addLoose(self, operation: EditOperation) -> bool:
        """
        Attempts to add the given `operation` such that it is loosely compatible with the overall `CompoundOperation`.
        This may change the type of the `CompoundOperation`. If the addition was successful, then returns True; else,
        returns False.
        """
        
        # turn the operation to be inserted into a compound operation, then get attibutes
        op = CompoundOperation(operation)
        opBeginIndex, opEndIndex = op.getIndexRange()
        opTokens = op.getTokens()

        # this is one past the index of the token last affected once this operation is applied
        endAffectedIndex = self.__beginIndex + len(self.__newTokens)

        # if the ranges of affected tokens do not touch, then quit
        if opEndIndex < self.__beginIndex or opBeginIndex > endAffectedIndex:
            return False
        
        # determine how this operation will change considering the added operation
        if opBeginIndex < self.__beginIndex:
            if opEndIndex <= endAffectedIndex:
                self.__newTokens = opTokens + self.__newTokens[opEndIndex - self.__beginIndex :]
                self.__beginIndex = opBeginIndex
            else:
                self.__newTokens = opTokens
                self.__beginIndex = opBeginIndex
                self.__endIndex += opEndIndex - endAffectedIndex
        else:
            if opEndIndex <= endAffectedIndex:
                self.__newTokens = (self.__newTokens[: opBeginIndex - self.__beginIndex] + opTokens +
                        self.__newTokens[opEndIndex - self.__beginIndex :])
            else:
                self.__newTokens = self.__newTokens[: opBeginIndex - self.__beginIndex] + opTokens
                self.__endIndex += opEndIndex - endAffectedIndex

        # set the type and return true
        self.__setType()
        return True
    
    def addStrict(self, operation: EditOperation) -> bool:
        """
        Attempts to add the given `operation` such that it is strictly compatible with the overall `CompoundOperation`.
        If the addition was successful, then returns True; else, returns False.
        """

        # stop if the operation is not of the same type
        if not (type(operation) is self.__type or 
                type(operation) is CompoundOperation and operation.getType() is self.__type):
            return False

        # add the operation
        return self.addLoose(operation)
    
    def applyToTokens(self, tokens: List[str]) -> None:
        """
        Applies the operation to the given list of tokens. Raises an error if it affects token indices which
        are out of bounds.
        """

        if self.__endIndex > len(tokens):
            raise IndexError("CompoundOperation: cannot delete index range {}:{} (out of bounds)".format(
                    self.__beginIndex, self.__endIndex))

        tokens[self.__beginIndex:self.__endIndex] = self.__newTokens
    
    def __setType(self) -> None:
        """
        Sets the type based on the index range and the tokens to be inserted.
        """

        indexRange = self.__endIndex - self.__beginIndex

        # If nothing is removed and nothing is inserted, then it's a replace operation
        if indexRange == 0 and len(self.__newTokens) == 0:
            self.__type = ReplaceOperation
        
        # If nothing is removed, then it's an insert operation
        elif indexRange == 0:
            self.__type = InsertOperation
        
        # if nothing is added, then it's a delete operation
        elif len(self.__newTokens) == 0:
            self.__type = DeleteOperation
        
        # anything else means that there are removals and additions, so it's a replaceOperation
        else:
            self.__type = ReplaceOperation

### Principles of CompoundOperations 

#### Type

Each `CompoundOperation` has a *type*, which is one of the following:

- `InsertOperation` -- indicates that no tokens are removed and at least one token is added
- `DeleteOperation` -- indicates that at least one token is removed and no tokens are added
- `ReplaceOperation` -- indicates one of the following:
    - no tokens are removed or added
    - at least one token is removed and at least one token is added  
    
#### Loose Compatibility

A squence of EditOperations is said to be *loosely compatible* if, when the operations are applied to an `AbstractMethod` directly after one another, the `AbstractMethod` is modified in one contiguous section. A `CompoundOperation` is loose when it consists of EditOperations that are loosely compatible with one another. Note that the order in which the EditOperations are applied matters. Take the following examples, which both utilize the same EditOperations:

In [ ]:
method = AbstractMethod("A B C D")
method.applyEditOperations([
    DeleteOperation(1),
    DeleteOperation(0)
])
method

C D

The two operations that were applied are loosely compatible because they modified (deleted) a contiguous section of tokens `['A', 'B']`.

In [ ]:
method = AbstractMethod("A B C D")
method.applyEditOperations([
    DeleteOperation(0),
    DeleteOperation(1)
])
method

B D

In this case, even though the same two operations were applied, the operations are not loosely compatible. The first operation deleted the token `'A'`, and the second token deleted the token `'C'`. Since these tokens were not contiguous, the applied operations are not loosely compatible.

#### Strict compatibility

A sequence of EditOperations is said to be *strictly compatible* if it is loosely compatible and all the operations are of the same type. A `CompoundOperation` is strict when it consists of EditOperations which are strictly compatible with one another.

These operations are strictly compatible:

In [ ]:
#hide_output
DeleteOperation(1)
DeleteOperation(1)

DELETE 1

These operations are not strictly compatible, even though they are loosely compatible:

In [ ]:
#hide_output
DeleteOperation(1)
InsertOperation(1, "foo")

INSERT 1 -> 'foo'

In [ ]:
#hide
from hephaestus.CondenseEditOperations import *

### Creating CompoundOperations

CompoundOperations are created from a sequence of EditOperations. The easiest way to do this is by using the utility functions `getCondensedBasic`, `getCondensedLoose`, and `getCondensedStrict` found in the `CondenseEditOperations` module. However, you can also create them manually by repeatedly adding EditOperations via the methods below:

In [ ]:
#hide_input
show_doc(CompoundOperation.addLoose)

<h4 id="CompoundOperation.addLoose" class="doc_header"><code>CompoundOperation.addLoose</code><a href="__main__.py#L89" class="source_link" style="float:right">[source]</a></h4>

> <code>CompoundOperation.addLoose</code>(**`operation`**:`Union`\[[`InsertOperation`](/hephaestus/EditOperations.html#InsertOperation), [`DeleteOperation`](/hephaestus/EditOperations.html#DeleteOperation), [`ReplaceOperation`](/hephaestus/EditOperations.html#ReplaceOperation), `ForwardRef('CompoundOperation')`\])

Attempts to add the given `operation` such that it is loosely compatible with the overall [`CompoundOperation`](/hephaestus/EditOperations.html#CompoundOperation).
This may change the type of the [`CompoundOperation`](/hephaestus/EditOperations.html#CompoundOperation). If the addition was successful, then returns True; else,
returns False.

In [ ]:
#hide_input
show_doc(CompoundOperation.addStrict)

<h4 id="CompoundOperation.addStrict" class="doc_header"><code>CompoundOperation.addStrict</code><a href="__main__.py#L129" class="source_link" style="float:right">[source]</a></h4>

> <code>CompoundOperation.addStrict</code>(**`operation`**:`Union`\[[`InsertOperation`](/hephaestus/EditOperations.html#InsertOperation), [`DeleteOperation`](/hephaestus/EditOperations.html#DeleteOperation), [`ReplaceOperation`](/hephaestus/EditOperations.html#ReplaceOperation), `ForwardRef('CompoundOperation')`\])

Attempts to add the given `operation` such that it is strictly compatible with the overall [`CompoundOperation`](/hephaestus/EditOperations.html#CompoundOperation).
If the addition was successful, then returns True; else, returns False.

In [ ]:
#export
#hide
# define for a second time here so that CompoundOperation no longer has to be a forward declaration
EditOperation = Union[InsertOperation, DeleteOperation, ReplaceOperation, CompoundOperation]